In [ ]:
! unzip my_archive.zip -d my_data

Archive:  my_archive.zip
 extracting: my_data/train/noun_verbtrain.json  
 extracting: my_data/train/noun_verb_nountrain.json  
replace my_data/train/noun_verb_nountrain.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
 extracting: my_data/train/noun_verb_nountrain.json  
 extracting: my_data/train/noun_verb_nountrain.json  
 extracting: my_data/train/noun_verb_nountrain.json  
 extracting: my_data/train/with_chtotrain.json  
 extracting: my_data/train/with_chtotrain.json  
 extracting: my_data/train/with_pptrain.json  
 extracting: my_data/train/with_pptrain.json  
 extracting: my_data/train/subj_reltrain.json  
 extracting: my_data/train/subj_reltrain.json  
 extracting: my_data/train/obj_reltrain.json  
 extracting: my_data/train/obj_reltrain.json  
 extracting: my_data/val/noun_verbval.json  
 extracting: my_data/val/noun_verb_nounval.json  
 extracting: my_data/val/noun_verb_nounval.json  
 extracting: my_data/val/noun_verb_nounval.json  
 extracting: my_data/val/noun_verb_nounval.js

In [ ]:
from transformers import AutoTokenizer, AutoModel
import json
from pathlib import Path
import itertools
from torch.utils.data.dataset import Dataset
import re

In [ ]:
!pip install jsonlines

In [ ]:
import jsonlines

In [ ]:
#  Test_dir = Path("/content/my_data/test")
#  Test_files = list(Test_dir.rglob("*.json"))
#  Train_dir = Path("/content/my_data/train")
#  Train_files = list(Train_dir.rglob("*.json"))
#  Val_dir = Path("/content/my_data/val")
#  Val_files = list(Val_dir.rglob("*.json"))

In [ ]:
MODES = ["train", "test", "val"]

In [ ]:
paths = {}
for mode in MODES:
    paths[f"{mode}_dir"] = Path(f"/content/my_data/{mode}")

paths

{'train_dir': PosixPath('/content/my_data/train'),
 'test_dir': PosixPath('/content/my_data/test'),
 'val_dir': PosixPath('/content/my_data/val')}

In [ ]:
def make_one_file(paths, mode):
    directory = paths[f"{mode}_dir"]
    files = list(directory.rglob("*.json"))
    output_file_path = f"{mode}_files.json"
    with open(output_file_path, "w", encoding = "utf-8") as output_file:
        for file in files:
            with open(file, "r", encoding = "utf-8") as input_file:
                content = input_file.read()
                output_file.write(content + "\n")

    with jsonlines.open(output_file_path, "r") as f:
        loaded = list(f)
        loaded = list(itertools.chain(*loaded))

    loaded_dict_list = []
    for elem in loaded:
        s_corrected = re.sub(r"(\w+):", r'"\1":', elem)

        d = json.loads(s_corrected)
        loaded_dict_list.append(d)
    return loaded_dict_list


In [ ]:
train_data = make_one_file(paths, MODES[0])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# Пример выполнения

In [ ]:
def preprocess(line):
    tokenized_data = tokenizer(line["sentence"])
    token_indexes = tokenized_data["input_ids"]
    tokens = tokenizer.convert_ids_to_tokens(token_indexes)
    return token_indexes

In [ ]:
train_data[15]

{'sentence': 'писатели были пожилыми', 'idx': 1, 'label': 'anim'}

In [ ]:
preprocess(train_data[15])

[101, 40862, 3294, 103659, 852, 102]

In [ ]:
tokenizer("[PAD]")["input_ids"][1]



0

In [ ]:
# Создание датасета

In [ ]:
class GrammarDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data
        self.tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")


    def __len__(self):
        return len(self.data)

    def preprocess(self, line):
        tokenized_data = self.tokenizer(line["sentence"])
        token_indexes = tokenized_data["input_ids"]

        return token_indexes, line["idx"]


    def __getitem__(self, idx):
        line = self.data[idx]
        token_indexes, target = self.preprocess(line)
        return {"input": token_indexes, "target": target}


In [ ]:
train_dataset = GrammarDataset(train_data)

In [ ]:
train_dataset[1]

{'input': [101, 13712, 49433, 2343, 102], 'target': 0}

**Датасет сформирован, теперь перейдем к формированию батчей, паддингу и прочему**

In [ ]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch

In [ ]:
batch_size = 10
num_epochs = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
def custom_padding(batch):
    inputs_list = [torch.tensor(item["input"]) for item in batch]
    targets_list = torch.tensor([torch.tensor(item["target"]) for item in batch])
    padded_inputs = pad_sequence(inputs_list, batch_first=True, padding_value=tokenizer("[PAD]")["input_ids"][1])
    return {"input": padded_inputs, "target": targets_list}

In [ ]:
# Проверка

# batch = [train_dataset[1], train_dataset[3], train_dataset[5]]
# custom_padding(batch)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_padding)

In [ ]:
# Проверка

it = iter(train_dataloader)
batch = next(it)
print(batch["input"].size())
# print(batch["target"])

torch.Size([10, 10])


**Модель + обучение**

In [ ]:
RUBERT = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased").to(device)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
answer = RUBERT(batch['input'])
# answer

In [ ]:
print(answer["last_hidden_state"].shape)
print(answer["pooler_output"].shape)

torch.Size([10, 10, 768])
torch.Size([10, 768])


In [ ]:
from transformers.optimization import AdamW
import torch.nn as nn

In [ ]:
class BasicTransformersClassificationModel(nn.Module):
    def __init__(self, model, label_num, learning_rate, device):
        super(BasicTransformersClassificationModel, self).__init__()
        self.model = model
        self.label_num = label_num
        self.criterion = nn.BCELoss(reduction="mean")
        self.device = device
        self.optimizer = AdamW(self.parameters(), lr=learning_rate, weight_decay=0.01)
        self.sigmoid = nn.Sigmoid()
        self.last_layer = torch.nn.Linear(self.model.config.hidden_size, label_num)

    @property
    def hidden_size(self):
        return self.model.config.hidden_size

    def train_batch(self, inp, targ):
        self.train()
        self.optimizer.zero_grad()
        loss = self.compute_loss(inp, targ)
        loss["loss"].backward()
        self.optimizer.step()
        return loss


    def compute_loss(self, inp, targ):
        targ = targ.to(self.device)
        logits = self(inp)
        loss = self.criterion(logits, targ)
        label = (logits >= 0.5).int()
        return {"loss": loss, "labels": label}

    def validate_on_batch(self, inp, targ):
        self.eval()
        with torch.no_grad():
            return self.compute_loss(inp, targ)


    def forward(self, inp):
        inp = torch.tensor(inp)
        if inp.dim() == 1:
            inp = inp.unsqueeze(0)
        input_ids = inp.to(self.device)


        cls_output = self.model(input_ids)["pooler_output"].to(self.device)
        output = self.last_layer(cls_output)
        logits = self.sigmoid(output)
        return logits




In [ ]:
model = BasicTransformersClassificationModel(RUBERT, label_num=1, learning_rate=1e-5, device=device).to(device)


In [ ]:
# for name, elem in model.named_parameters():
#     print(name, elem.device, elem.shape)

In [ ]:
for epoch in range(num_epochs):
    for batch in train_dataloader:
        input_ids = batch["input"].to(device)
        targets = batch["target"].unsqueeze(-1).float().to(device)
        loss = model.train_batch(input_ids, targets)["loss"].item()
    print(epoch, loss)

print(model.validate_on_batch(input_ids, targets)["loss"].item())

<ipython-input-97-45ec3a382c17>:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inp = torch.tensor(inp)


0 0.0015078316209837794
1 0.0006982173654250801
2 0.00032578958780504763
3 0.0001901125506265089
4 0.00022589355648960918
5 8.737462485441938e-05
6 7.369978266069666e-05
7 5.090588456369005e-05
8 3.8124311686260626e-05
9 2.960700476251077e-05
1.9436285583651625e-05


In [ ]:

# def predict_with_model(model, X, classes):
#     model.eval()
#     dataloader = DataLoader(X, batch_size=8, shuffle=False)
#     answer = [None] * len(X)
#     for batch in dataloader:
#         input_ids = batch["input"].to(model.device)
#         with torch.no_grad():
#             batch_answer = model(input_ids)
#         labels = (batch_answer >= 0.5).int().cpu().numpy()
#         for i, label in zip(batch["indexes"], labels):
#             answer[i] = classes[label]
#     return answer

In [ ]:
# line = test_data[19]
# line

In [ ]:
# tokenized_data = tokenizer(line["sentence"])
# token_indexes = tokenized_data["input_ids"]

In [ ]:
# метод предсказывает класс из тестовой выборки

def predict_class(model, index_of_test_sentence):
    model.eval()
    data = test_data[index_of_test_sentence]["sentence"]
    tokenized_data = tokenizer(data)
    input_ids = tokenized_data["input_ids"]
    with torch.no_grad():
        batch_answer = model(input_ids)
    label = (batch_answer >= 0.5).int().cpu()
    return label.item()

In [ ]:
predict_class(model, 19)

0

In [ ]:
# Пример работы

In [ ]:
def predict_class_for_random_text(model, text):
    model.eval()
    data = text
    tokenized_data = tokenizer(data)
    input_ids = tokenized_data["input_ids"]
    with torch.no_grad():
        batch_answer = model(input_ids)
    label = (batch_answer >= 0.5).int().cpu()
    return label.item()

In [ ]:
text = "человек ходить"
text1 = "человек ходит"
text3 = "верблюд были горбатым"
text4 = "верблюд был горбатым"
text5 = "смеялся возле комики"
text6 = "смеялся возле комика"
text7 = "воспитатель который забыли"
text8 = "воспитатель который забыл"


In [ ]:
predict_class_for_random_text(model, text8)

1

In [ ]:
# test_data = make_one_file(paths, MODES[1])
# test_dataset = GrammarDataset(test_data)
# predicted = predict_with_model(model, test_dataset, classes=[0, 1])

In [ ]:
test_data[0]

{'sentence': 'рассказ от секретаря был новым', 'idx': 1, 'label': 'pp_inan'}